### Noms et Prénoms du binome :
- Hadrien SALEM
- Emilie SALEM
---

# TP en Watermarking - TP2

---

## Patrick Bas, CNRS, CRIStAL

---


 

## Tatouage par également de spectre et attaques de sécurité

### 2.1 Notations et rappels:

Les notations sont identiques à celles vues en cours. Le procédé d'insertion est le schéma par étalement de spectre vu en cours.

* X = matrice de $N_{i}$ vecteurs originaux de taille $N_{v}$ ($N_{i}$ colonnes, $N_{v}$ lignes). $N_{i}$ représente par exemple le nombre de contenu traités, et $N_{v}$ le nombre de composantes tatouées par contenu. **Note**: chaque colonne de X peut par exemple représenter des composantes d'une image.
* $N_{o}$ représente le nombre de contenus tatoués observés par l'adversaire et utilisés pour construire son attaque
* $N_{i}$ représente le nombre de contenus tatoués utilisés pour calculer pratiquement le taux d'erreur (voir BER)
* Y = matrice de contenus tatoués
* Z = matrice de contenus tatoués et perturbés
* k clé secrète de norme unitaire
* $m_{1}$: bit inséré, converti en +1, -1 
* $\alpha$: paramètre de distorsion
* BER: Bit Error Rate, taux d'erreur binaire ou encore probabilité d'erreur empirique de décodage
* DWR: « Document to Watermark Ratio » $DWR=10\log_{10}(\sum x_{i}^{2}/\sum w_{i}^{2})$, permet de mesurer la distorsion ($DWR=0$ $\Leftrightarrow$ $\sigma_{X}^{2}=\sigma_{W}^{2}$ ). Permet de mesurer la distortion. Distortion nulle $DWR=\infty$, distortion importante $DWR \rightarrow 0$ 

#### Rappels:
* L'objectif du récepteur est de bien décoder $m_{1}$, possiblement en ayant une distortion qui ne soit pas trop importante
* Ici, les objectifs de l'adversaire sont d'estimer la clé k puis d'effacer le message inséré. Pour s'assurer que l'adversaire a bien réussi à estimer la clé, il calculera la correlation normalisée entre le vecteur k et son estimation.

**N'oubliez pas d'exporter votre TP en html lors de sa remise**


In [ ]:
%matplotlib inline  

import numpy as np
import matplotlib.pyplot as plt
from scipy.special import erf
from sklearn.decomposition import FastICA

## Scénario 1: Attaque à Messages connus, 1 bit
* Mise en route: Quel est le BER cible de l'adversaire ?
* Mettre en place l'attaque liée à ce scénario
* Etudier l'impact de $N_{o}$ et de $\alpha$ sur le BER après attaque

## Réponses 

- Le BER cible de l'adversaire est de $0.5$ : en effet, si le contenu n'est pas tatoué, on fait une erreur sur deux lors du décodage du message. Autrement dit, on ne peut plus retrouver le message inséré (puisque le message inséré a été effacé).
- Voir code ci-dessous, en particulier l'implémentation de `hatk` en utilisant la formule : 
  $$
  \hat k_1 = \frac{1}{\alpha N_o} \sum - (-1)^m y_i
  $$
- Comme on peut le voir sur la courbe "`BER après attaque en fonction du nombre No d'observations utilisées`", plus $N_o$ augmente plus le BER se rapproche de la valeur cible $0.5$. En effet, plus on fait d'observations, mieux on est capable d'estimer la valeur de $k$ (moyenne empirique) ce qui permet de retirer le message $m$ efficacement.
- La courbe "`BER après attaque en fonction du coefficient alpha`" montre que plus le coefficient $\alpha$ augmente, plus on semble se rapprocher et osciller autour de la valeur cible $0.5$. En effet, plus le coefficient de distorsion est important, plus il est facile de déterminer la direction correspondant à la clé (vecteurs bien polarisés).

In [ ]:
Nv = 100 # Size of the vector
Ni = 10000 # Max number of observations

In [ ]:
def ber(Y,m,k):# Compute the Bit Error Rate between message m and the extracted message from Y using key k
    c = np.sign(np.dot(Y.T,k))
    return np.sum(c != m)/np.float(Ni)

def norm_corr(hatk,k): # Correlation between k and its estimate
    hatk = hatk / np.sqrt(np.dot(hatk.T,hatk)) # Normalize
    corrN = np.abs(np.dot(hatk.T,k))/(np.linalg.norm(hatk)*np.linalg.norm(k)) # Compute the Normalised correlation
    return corrN

def do_process(alpha,No):
    
    print('alpha: ',alpha)
    print('No: ',No)

    X = np.random.randn(Nv,Ni) # Generate Ni random host vectors
    k = np.random.randn(Nv,1) # Generate the Watermark
    k = k / np.sqrt(np.dot(k.T,k)) # Normalize the watermark

    m1 = np.ones((Ni,1)) # Scenario with Known Messages: generate only ones, to be changed for the WOA attack!!!
    K = np.dot(k,m1.T) # Generate the matrix of watermarks (each column contains m1_i*k)
    W = alpha*K
    Y = X + W # perform embedding
    DWR = 10*np.log10(Nv/alpha**2) # Set the Document to Watermark Ratio, in dB
    print('DWR: ',DWR,' dB')

    cY = np.sign(np.dot(Y.T,k)) # Computation of the decoded 'bits' (here -1 or +1)
    print('practical bit error rate:')
    print(np.sum(cY != m1)/np.float(Ni)) 

    # Attack
    Y_obs = Y[:,:No]
    
    hatk = np.zeros(Nv)
    for v in range(Nv): # For each component of the key
        for i in range(No): # Sum on observations
            hatk[v] += -((-1)**m1[i])*Y_obs[v][i]/(alpha*No)
    hatk = hatk / np.sqrt(np.dot(hatk.T,hatk)) # We need to Normalize
    
    corrN = np.abs(np.dot(hatk.T,k))/(np.linalg.norm(hatk)*np.linalg.norm(k)) # Compute the Normalised correlation
    print('Normalised correlation between the true key and the estimated key')
    print(corrN)

    hatk = np.reshape(hatk,(Nv,1)) # We need to reshape

    YA = Y - alpha*np.dot(hatk,m1.T) # KMA: perform the removal attack
    practical_ber = ber(YA,m1,k)
    print('practical bit error rate after security attack')
    print(practical_ber)
    print('\n')
    return practical_ber, corrN
    

alpha = 2 # Tune the power of the watermark here

ber_list_No = []
corrN_list_No = []
No_list = np.arange(100,2000,100)

for No in No_list:
    practical_ber, corrN = do_process(alpha,No)
    ber_list_No.append(practical_ber)
    corrN_list_No.append(corrN)

In [ ]:
plt.figure(figsize=(20,7))

plt.subplot(121)
plt.title("BER après attaque en fonction du nombre No d'observations utilisées")
plt.xlabel("Nombre d'observations No")
plt.ylabel("BER (en bits)")
plt.plot(No_list, ber_list_No)

plt.subplot(122)
plt.title("Corrélation normalisée après attaque en fonction du nombre No d'observations utilisées")
plt.xlabel("Nombre d'observations No")
plt.ylabel("Corrélation normalisée")
plt.plot(No_list, corrN_list_No)

In [ ]:
ber_list_alpha = []
corrN_list_alpha = []
alpha_list = np.arange(1,20)

for alpha in alpha_list:
    practical_ber, corrN_alpha = do_process(alpha,No=1000)
    ber_list_alpha.append(practical_ber)
    corrN_list_alpha.append(corrN_alpha)

In [ ]:
plt.figure(figsize=(20,7))

plt.subplot(121)
plt.title("BER après attaque en fonction du coefficient alpha")
plt.xlabel("Coefficient alpha")
plt.ylabel("BER (en bits)")
plt.plot(alpha_list, ber_list_alpha)

plt.subplot(122)
plt.title("Corrélation normalisée après attaque en fonction du nombre No d'observations utilisées")
plt.xlabel("Coefficient alpha")
plt.ylabel("Corrélation normalisée")
plt.plot(alpha_list, corrN_list_alpha)

## Scénario 2, Attaque à messages inconnus, 1bit
* Note: la fonction `np.linalg.eig` peut être utilisée pour effectuer une décomposition en valeurs et vecteurs propres.
* Note: pour effacer le message inséré, il conviendra au préalable d'estimer le bit inséré, cela peut se faire via `m_est = np.sign(np.dot(Y.T,hatk))` où `hatk` est la clé estimée
* Mettre en place l'attaque
* Etudier l'impact de $N_{o}$ et de $\alpha$ sur le BER après attaque

## Réponses

* Encore une fois, plus on augmente le nombre d'observations $N_{o}$, plus on se rapproche de la valeur de BER cible $0.5$. Cependant, contrairement au premier scénario d'attaque, il faut plus d'observations pour s'en rapprocher, et même comme cela le résultat est moins bon (plus proche de $0.47$ que de $0.5$). En effet, on ne connaît plus le message ce qui le rend plus difficile à trouver, et nécessite d'inférer la direction de plus grande variance en se basant sur des observations.
* De même, plus on augmente le coefficient $\alpha$, plus le BER se rapproche de $0.5$ pour la même raison qu'au scénario précédent. En effet, lorsque la distortion est importante, les vecteurs sont très polarisés, ce qui rend plus facile la détermination de la direction de plus grande variance par la PCA.

In [ ]:
plt.figure(figsize=(20,7))

plt.subplot(121)
plt.title("BER après attaque en fonction du nombre No d'observations utilisées")
plt.xlabel("Nombre d'observations No")
plt.ylabel("BER (en bits)")
plt.plot(No_list, ber_list_No)

plt.subplot(122)
plt.title("Corrélation normalisée après attaque en fonction du nombre No d'observations utilisées")
plt.xlabel("Nombre d'observations No")
plt.ylabel("Corrélation normalisée")
plt.plot(No_list, np.ravel(corrN_list_No))

In [ ]:
# 2dn scenario, scenario with KMA 
print('2nd Scenario')

def do_process(alpha,No):
    
    print('alpha: ',alpha)
    print('No: ',No)

    X = np.random.randn(Nv,Ni) # Generate Ni random host vectors
    k = np.random.randn(Nv,1) # Generate de Watermark
    k = k / np.sqrt(np.dot(k.T,k)) # Normalize the watermark

    m1 = np.sign(np.random.randn(Ni,1)) #Scenario with unknow messages, first bit

    K = np.dot(k,m1.T) # Generate the matrix of watermarks (each column contains m1_i*k)

    W = alpha*K

    Y = X + W # perform embedding

    # Attack
    Y_obs = Y[:,:No]

    cov = np.cov(Y_obs)
    eigvals, eigvects = np.linalg.eig(cov)

    idx = eigvals.argsort()[::-1]   
    eigvals = eigvals[idx]
    eigvects = eigvects[:,idx]
    
    hatk = eigvects[:,0]
    hatk = np.reshape(hatk,(Nv,1)) # You might need to reshape the estimated key
    corrN = norm_corr(hatk,k) # To ease the writing we use the norm_corr function
    print(f'Normalised correlation between the true key and the estimated key: {corrN[0][0]}')

    m_est = np.sign(np.dot(Y.T,hatk))
    YA = Y - alpha*np.dot(hatk,m_est.T) # KMA: perform the removal attack

    print(f'bit error rate after security attack: {ber(YA,m1,k)}')
    print('\n')
    return practical_ber, corrN


    
alpha = 2 # Tune the power of the watermark here

ber_list_No = []
corrN_list_No = []
No_list = np.arange(100,2000,100)

for No in No_list:
    practical_ber, corrN = do_process(alpha,No)
    ber_list_No.append(practical_ber)
    corrN_list_No.append(corrN)

In [ ]:
ber_list_alpha = []
corrN_list_alpha = []
alpha_list = np.arange(1,20)

for alpha in alpha_list:
    practical_ber, corrN_alpha = do_process(alpha,No=1000)
    ber_list_alpha.append(practical_ber)
    corrN_list_alpha.append(corrN_alpha)

In [ ]:
ber_list_alpha = []
corrN_list_alpha = []
alpha_list = np.arange(1,20)

for alpha in alpha_list:
    practical_ber, corrN_alpha = do_process(alpha,No=1000)
    ber_list_alpha.append(practical_ber)
    corrN_list_alpha.append(corrN_alpha)

In [ ]:
plt.figure(figsize=(20,7))

plt.subplot(121)
plt.title("BER après attaque en fonction du coefficient alpha")
plt.xlabel("Coefficient alpha")
plt.ylabel("BER (en bits)")
plt.plot(alpha_list, ber_list_alpha)

plt.subplot(122)
plt.title("Corrélation normalisée après attaque en fonction du nombre No d'observations utilisées")
plt.xlabel("Coefficient alpha")
plt.ylabel("Corrélation normalisée")
plt.plot(alpha_list, np.ravel(corrN_list_alpha))

## Scénario 3: Attaque à messages inconnus, 2 bits
* Vérifier que l'attaque précédente ne permet pas d'estimer les deux clés. 
* Estimer au moins l'une des clés utilisée
    * Note: on pourra utiliser l'algorithme `FastICA` pour estimer les deux composantes indépendantes 
    * Pour cela on pourra appeler la fonction fastica en spécifiant que l'analyse en composantes indépendantes s'effectuera sur un sous espace engendré par les *deux premières composantes principales* (`n_components=2`), et en récupérant les colonnes de la matrice de mélange A (obtenue via `ica.mixing_`) estimé par l'algorithme.
    * Vérifier, à l'aide de la corrélation normalisée, que cette méthode permet d'estimer la clé `k1`
* Mettre en place l'attaque qui permet d'effacter un bit sur les deux
* Etudier l'impact de $N_{o}$ et de $\alpha$ sur le BER après attaque

## Réponse

* Nous avons essayé d'appliquer la méthode du scénario 2 (PCA) pour la détermination de la clé *k1*: on observe que les corrélations normalisées ont des valeurs avoisinant 70%, ce qui n'est pas du tout satisfaisant: l'attaque précédente ne permet donc pas d'estimer les deux clés.
* Les courbes ci-dessous montrent qu'avec la méthode ICA, on est bien capable de d'estimer les deux composantes indépendantes. En particulier, pour la clé `k1`, on obtient des BER proches de 0.5 et des corrélations normalisées proches de 1, ce qui confirme l'efficacité de la méthode.
* L'impact de $N_o$ et de $\alpha$ est le même ici que pour les deux autres scénarios: le nombre d'observations améliore la qualité de la solution et le coefficient alpha rend plus simple la détermination des directions indépendantes.

In [ ]:
# 3rd scenario, 2 bits
print('3rd scenario, 2bits')

m1 = np.sign(np.random.randn(Ni,1))#Scenario with unknow messages, first bit
m2 = np.sign(np.random.randn(Ni,1))#Scenario with unknow messages, second bit

k1 = np.random.randn(Nv,1) # Generate de Watermark
k1 = k1 / np.sqrt(np.dot(k1.T,k1)) # Normalize the watermark

k2 = np.random.randn(Nv,1) # Generate de Watermark
k2 = k2 / np.sqrt(np.dot(k2.T,k2)) # Normalize the watermark

In [ ]:
# 3rd scenario,n 2 bits, but applying method of scenario 2
def do_process_with_old_method(alpha,No):

    X = np.random.randn(Nv,Ni) # Generate Ni random host vectors
    K = np.dot(k1,m1.T) + np.dot(k2,m2.T) # Generate the matrix of watermarks (each column contains m1_i*k)
    W = alpha*K
    Y = X + W # perform embedding
    Y_obs = Y[:,:No]

    cov = np.cov(Y_obs)
    eigvals, eigvects = np.linalg.eig(cov)

    idx = eigvals.argsort()[::-1]   
    eigvals = eigvals[idx]
    eigvects = eigvects[:,idx]
    
    hatk_1 = eigvects[:,0]
    hatk_2 = eigvects[:,1]

    hatk_1 = hatk_1/np.linalg.norm(hatk_1) # Normalize the vector
    hatk_1 = np.reshape(hatk_1,(Nv,1)) # Reshape for upcoming comparisons

    hatk_2 = hatk_2/np.linalg.norm(hatk_2)
    hatk_2 = np.reshape(hatk_2,(Nv,1))

    # It is uncertain which key hatk_1 and hatk_2 correspond to, so we need to test
    corrN_v1 = norm_corr(hatk_1,k1) # Compute the Normalised correlation
    corrN_v2 = norm_corr(hatk_1,k2)
    
    if(corrN_v1 < corrN_v2) : hatk_1, hatk_2 = hatk_2, hatk_1

    corrN_1 = norm_corr(hatk_1,k1)
    print(f'Normalised correlation between the estimated key and k1: {corrN_1[0][0]}')

    corrN_2 = norm_corr(hatk_2,k2) # Compute the Normalised correlation
    print(f'Normalised correlation between the estimated key and k2: {corrN_2[0][0]}')

    m1_est = np.sign(np.dot(Y.T,hatk_1))
    m2_est = np.sign(np.dot(Y.T,hatk_2))
    
    YA_1 = Y - alpha*np.dot(hatk_1,m1_est.T) # KMA: perform the removal attack
    YA_2 = Y - alpha*np.dot(hatk_2,m2_est.T)
    
    ber1 = ber(YA_1,m1,k1)
    ber2 = ber(YA_2,m2,k2)
    print(f'bit error rate after security attack for the first bit: {ber1}')
    print(f'bit error rate after security attack for the second bit: {ber2}')
    print('\n\n')

# One example, can be used to draw plots
for No in range(100,2000,100):
    do_process_with_old_method(alpha,No)

In [ ]:
def do_process(alpha,No):

    print('alpha: ',alpha)
    print('No: ',No)

    X = np.random.randn(Nv,Ni) # Generate Ni random host vectors

    K = np.dot(k1,m1.T) + np.dot(k2,m2.T) # Generate the matrix of watermarks (each column contains m1_i*k)

    W = alpha*K

    Y = X + W # perform embedding
    
    Y_obs = Y[:,:No]

    ica = FastICA(n_components=2)
    ica.fit(Y_obs.T)

    hatk_1 = ica.mixing_[:,1]
    hatk_2 = ica.mixing_[:,0]
    
    hatk_1 = hatk_1/np.linalg.norm(hatk_1) # Normalize the vector
    hatk_1 = np.reshape(hatk_1,(Nv,1)) # Reshape for upcoming comparisons

    hatk_2 = hatk_2/np.linalg.norm(hatk_2)
    hatk_2 = np.reshape(hatk_2,(Nv,1))

    # It is uncertain which key hatk_1 and hatk_2 correspond to, so we need to test
    corrN_v1 = norm_corr(hatk_1,k1) # Compute the Normalised correlation
    corrN_v2 = norm_corr(hatk_1,k2)
    
    if(corrN_v1 < corrN_v2) : hatk_1, hatk_2 = hatk_2, hatk_1

    corrN_1 = norm_corr(hatk_1,k1)
    print(f'Normalised correlation between the estimated key and k1: {corrN_1[0][0]}')

    corrN_2 = norm_corr(hatk_2,k2) # Compute the Normalised correlation
    print(f'Normalised correlation between the estimated key and k2: {corrN_2[0][0]}')

    YA_1 = np.zeros((Nv,Ni)) # Perform the attack

    m1_est = np.sign(np.dot(Y.T,hatk_1))
    m2_est = np.sign(np.dot(Y.T,hatk_2))
    
    YA_1 = Y - alpha*np.dot(hatk_1,m1_est.T) # KMA: perform the removal attack
    YA_2 = Y - alpha*np.dot(hatk_2,m2_est.T)
    
    ber1 = ber(YA_1,m1,k1)
    ber2 = ber(YA_2,m2,k2)
    print(f'bit error rate after security attack for the first bit: {ber1}')
    print(f'bit error rate after security attack for the second bit: {ber2}')
    print('\n\n')
    # We only return the BER and correlation for the first bit
    return ber1, corrN_1[0][0]

alpha = 2 # Tune the power of the watermark here

ber_list_No = []
corrN_list_No = []
No_list = np.arange(100,2000,100)

# One example, can be used to draw plots
for No in range(100,2000,100):
    practical_ber, corrN = do_process(alpha,No)
    ber_list_No.append(practical_ber)
    corrN_list_No.append(corrN)

In [ ]:
plt.figure(figsize=(20,7))

plt.subplot(121)
plt.title("BER du bit1 après attaque en fonction du nombre No d'observations utilisées")
plt.xlabel("Nombre d'observations No")
plt.ylabel("BER (en bits)")
plt.plot(No_list, ber_list_No)

plt.subplot(122)
plt.title("Corrélation normalisée du bit1 après attaque en fonction du nombre No d'observations utilisées")
plt.xlabel("Nombre d'observations No")
plt.ylabel("Corrélation normalisée")
plt.plot(No_list, np.ravel(corrN_list_No))

In [ ]:
ber_list_alpha = []
corrN_list_alpha = []
alpha_list = np.arange(1,20)

for alpha in alpha_list:
    practical_ber, corrN_alpha = do_process(alpha,No=1000)
    ber_list_alpha.append(practical_ber)
    corrN_list_alpha.append(corrN_alpha)

In [ ]:
plt.figure(figsize=(20,7))

plt.subplot(121)
plt.title("BER du bit1 après attaque en fonction du coefficient alpha")
plt.xlabel("Coefficient alpha")
plt.ylabel("BER (en bits)")
plt.plot(alpha_list, ber_list_alpha)

plt.subplot(122)
plt.title("Corrélation normalisée du bit1 après attaque en fonction du nombre No d'observations utilisées")
plt.xlabel("Coefficient alpha")
plt.ylabel("Corrélation normalisée")
plt.plot(alpha_list, corrN_list_alpha)

## Pour conclure: Etude de la robustesse
* Calculer le taux d'erreur (BER pour Bit Error Rate) théorique après ajout de bruit (voir cours)
* Etudier l'évolution de la robustesse (via le BER) en fonction de la distortion $\alpha$ 
* Quel compromis observe-t-on entre la sécurité et la robustesse?

# Stéganalyse par apprentissage
## Mise en route:
* Récupèrer les caractéristiques ici: https://nextcloud.univ-lille.fr/index.php/s/i6xr4JykqAASapN
* On charge les caractéristiques extraites à partir des images Cover et Stego pour d=3 (dimension de l'histogramme multivarié) et T=3 (seuil)

In [ ]:
cover = np.loadtxt('Features/cover-spam-N=3-T=3.csv')
stego = np.loadtxt('Features/stego-0.20-lsb-spam-N=3-T=3.csv')

In [ ]:
print(cover.shape)
print(stego.shape)

**Quelle est la dimension des caractéristiques ? Pourquoi ?**

On a 10000 échantillons respectifs d'images cover et stégo contenant chacun les 686 bins d'un histogramme multivarié associé à une image. On a par conséquent 686 caractéristiques.

**Entrainer un classifieur linéaire avec 5000 images en apprentissage et en test (effectuer une permutation pseudo-aléatoire des images avant l'apprentissage)**

In [ ]:
# Creating the train set
# Label 0 is for cover images and label 1 is for stego images
cover_train = np.hstack([np.zeros(5000).reshape((5000,1)),cover[:5000,:]]) # We label 5000 cover images examples...
stego_train = np.hstack([np.ones(5000).reshape((5000,1)),stego[:5000,:]]) # ... and 5000 stego images examples
train = np.vstack([cover_train, stego_train]) # Cover and stego images are put together
np.random.shuffle(train) # Shuffle the data

# Separating train data and labels
t_train = train[:, 0]
data_train = train[:,1:]

# Creating the test set
cover_test = np.hstack([np.zeros(5000).reshape((5000,1)),cover[5000:,:]])
stego_test = np.hstack([np.ones(5000).reshape((5000,1)),stego[5000:,:]])
test = np.vstack([cover_test, stego_test])
np.random.shuffle(test)

# Separating test data and labels
t_test = test[:,0]
data_test = test[:,1:]

In [ ]:
# Discriminant Analysis (LDA linear classification, QDA for non-linear classification)
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

lda = LinearDiscriminantAnalysis().fit(data_train, t_train)
qda = QuadraticDiscriminantAnalysis().fit(data_train, t_train)

score_lda = lda.score(data_test, t_test)
score_qda = qda.score(data_test, t_test)

print(f"Avec un classifieur linéaire (LDA), on obtient un score de {score_lda}.")
print(f"Avec un classifieur non-linéaire (QDA), on obtient un score de {score_qda}.")

## TODO: Commentaire

**Effectuer plusieurs entrainements/test successifs sur des ensembles d'apprentissage et de test différents (permutations différentes), commentez la variabilité**

**Comparer avec les caractéristiques produites pour N = 2 et T = 4 (fournies), expliquer la différence de performance**